# Тестирование Спарсификации Conv2D: Original vs Custom с анализом производительности

Notebook для сравнительного анализа спарсификации оригинальной реализации Conv2D и кастомной реализации с использованием Img2Col трансформации.

**Методы спарсификации:**
- MagnitudePruner (структурная и неструктурная)
- IterativePruner (структурная и неструктурная)

In [1]:
# pip uninstall torch torchvision torchaudio -y
# pip cache purge
# pip install --pre torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/nightly/cu130

## 1. Подготовка окружения

In [12]:
import os
import sys
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from tqdm import tqdm
import json
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

print(f'PyTorch версия: {torch.__version__}')
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))
print(torch.cuda.get_device_capability(0))
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Используемое устройство: {DEVICE}')

PyTorch версия: 2.10.0.dev20251118+cu130
True
NVIDIA GeForce RTX 5080
(12, 0)
Используемое устройство: cuda


## 2. Импорт компонентов проекта

In [13]:
project_root = Path(os.getcwd()).parent
sys.path.insert(0, str(project_root))
sys.path.insert(0, str(project_root / 'experiments'))
sys.path.insert(0, str(project_root / 'src'))


from src.models.camvid_segmentation_multiclass import (
    get_dataloaders,
    CamVidModel,
    visualize_sample,
    visualize_data,
    train_val,
    save_load_torch_model,
)

from experiments.common.replace_conv_resnet import replace_conv2d_with_custom

from experiments.prune import MagnitudePruner, IterativePruner

from src.core.latency import (
    latency_cpu,
    latency_gpu,
    latency_cpu_profiler,
    latency_gpu_event,
)

from src.utils import (
    basicconv_model,
    custom_conv_model,
    compare_models,
    analyze_benchmark_data,
)

from src.core.metric_writer import LatencyMetricsWriter

print('Все компоненты успешно загружены')

Все компоненты успешно загружены


## 3. Конфигурация

In [14]:
PROJECT_ROOT = os.getcwd()
DATA_DIR = os.path.join(PROJECT_ROOT, 'data', 'CamVid')

x_train_dir = os.path.join(DATA_DIR, "train")
y_train_dir = os.path.join(DATA_DIR, "trainannot")
x_valid_dir = os.path.join(DATA_DIR, "val")
y_valid_dir = os.path.join(DATA_DIR, "valannot")
x_test_dir = os.path.join(DATA_DIR, "test")
y_test_dir = os.path.join(DATA_DIR, "testannot")

print(os.listdir(DATA_DIR))

WEIGHTS_DIR = 'weights'
os.makedirs(WEIGHTS_DIR, exist_ok=True)

ORIGINAL_MODEL_PATH = os.path.join(WEIGHTS_DIR, 'originalConv_camvid_model_fp32.pt')
CUSTOM_MODEL_PATH = os.path.join(WEIGHTS_DIR, 'replacedConv_camvid_model_fp32.pt')

config = {
    'batch_size': 8,
}

pruning_configs = [
    # {'method': 'MagnitudePruner', 'sparsity': 0.5, 'structured': False, 'epochs': 5},
    # {'method': 'MagnitudePruner', 'sparsity': 0.5, 'structured': True, 'epochs': 5},
    # {'method': 'IterativePruner', 'sparsity': 0.5, 'structured': False, 'epochs': 5},
    {'method': 'IterativePruner', 'sparsity': 0.5, 'structured': True, 'epochs': 8},
]

bs_range = [1, 2, 4, 8, 16, 32]

['train', 'train.txt', 'trainannot', 'testannot', 'valannot', 'val', 'val.txt', 'test', 'test.txt']


## 4. Загрузка датасета

In [15]:
print('Загружаем датасет CamVid...')

try:
    train_loader, valid_loader, test_loader = get_dataloaders(
        x_train_dir, y_train_dir, x_valid_dir, y_valid_dir, x_test_dir, y_test_dir, bs=config['batch_size']
    )
    print(f'Датасет загружен')
except Exception as e:
    print(f'Ошибка: {e}')
    train_loader = []
    valid_loader = []
    test_loader = []

Загружаем датасет CamVid...
Датасет загружен


## 5. Загрузка предобученных моделей

In [16]:
print('Загружаем модели...')

OUT_CLASSES = len(train_loader.dataset.CLASSES)
original_model = CamVidModel("FPN", "resnext50_32x4d", in_channels=3, out_classes=OUT_CLASSES).to(DEVICE)
if os.path.exists(ORIGINAL_MODEL_PATH):
    original_model.load_state_dict(torch.load(ORIGINAL_MODEL_PATH, map_location=DEVICE))
    print(f'Original модель загружена')
else:
    print(f'Original модель не найдена')

original_model.eval()

custom_model = CamVidModel("FPN", "resnext50_32x4d", in_channels=3, out_classes=OUT_CLASSES).to(DEVICE)
replace_conv2d_with_custom(custom_model)
if os.path.exists(CUSTOM_MODEL_PATH):
    custom_model.load_state_dict(torch.load(CUSTOM_MODEL_PATH, map_location=DEVICE))
    print(f'Custom модель загружена')
else:
    print(f'Custom модель не найдена')

custom_model.eval()
print(f'Обе модели готовы')

Загружаем модели...
Original модель загружена
Custom модель загружена
Обе модели готовы


## 6. Спарсификация и Fine-tuning

In [17]:
pruned_models = {}

print('='*80)
print('СПАРСИФИКАЦИЯ И FINE-TUNING')
print('='*80)

for config_idx, pruning_config in enumerate(pruning_configs):
    method = pruning_config['method']
    sparsity = pruning_config['sparsity']
    structured = pruning_config['structured']
    epochs = pruning_config['epochs']
    
    struct_str = 'structured' if structured else 'unstructured'
    method_str = method.replace('Pruner', '').lower()
    
    print(f'\nМетод: {method} ({struct_str})')
    
    # Original модель
    print(f'Original Conv2d:')
    
    if method == 'MagnitudePruner':
        pruner = MagnitudePruner(sparsity=sparsity, structured=structured)
        orig_model = CamVidModel("FPN", "resnext50_32x4d", in_channels=3, out_classes=OUT_CLASSES).to(DEVICE)
        orig_model.load_state_dict(original_model.state_dict())
        orig_model.train()
        orig_model = pruner.apply_pruning(orig_model)
    else:
        pruner = IterativePruner(
            initial_sparsity=0.1,
            final_sparsity=sparsity,
            num_iterations=epochs,
            epochs_per_iteration=3,
            structured=structured
        )
        orig_model = CamVidModel("FPN", "resnext50_32x4d", in_channels=3, out_classes=OUT_CLASSES, iterative_pruner=pruner).to(DEVICE)
        orig_model.load_state_dict(original_model.state_dict())
        orig_model.train()
    
    if len(train_loader) > 0:
        valid_metrics, test_metrics, _ = train_val(
            orig_model,
            train_loader,
            valid_loader,
            test_loader,
            max_epochs=epochs,
            test=True,
        )
    else:
        valid_metrics = [{'valid_dataset_iou': None}]
        test_metrics = [{'test_dataset_iou': None}]
    
    orig_model.eval()
    orig_save_path = os.path.join(
        WEIGHTS_DIR,
        f'originalConv_{method_str}_{struct_str}_camvid_model_sp0{round(sparsity*10)}_fp32.pt'
    )
    save_load_torch_model(orig_model, orig_save_path, save=True)
    print(f'  Сохранено: {orig_save_path}')
    pruned_models[f'{method_str}_{struct_str}_sp0{round(sparsity*10)}'] = (orig_model, valid_metrics, test_metrics)

    # Custom модель
    print(f'Custom Img2Col:')
    
    if method == 'MagnitudePruner':
        pruner = MagnitudePruner(sparsity=sparsity, structured=structured)
        cust_model = CamVidModel("FPN", "resnext50_32x4d", in_channels=3, out_classes=OUT_CLASSES).to(DEVICE)
        replace_conv2d_with_custom(cust_model)
        cust_model.load_state_dict(custom_model.state_dict())
        cust_model.train()
        cust_model = pruner.apply_pruning(cust_model)
    else:
        pruner = IterativePruner(
            initial_sparsity=0.1,
            final_sparsity=sparsity,
            num_iterations=epochs,
            epochs_per_iteration=3,
            structured=structured
        )
        cust_model = CamVidModel("FPN", "resnext50_32x4d", in_channels=3, out_classes=OUT_CLASSES, iterative_pruner=pruner).to(DEVICE)
        replace_conv2d_with_custom(cust_model)
        cust_model.load_state_dict(custom_model.state_dict())
        cust_model.train()
    
    if len(train_loader) > 0:
        valid_metrics, test_metrics, _ = train_val(
            cust_model,
            train_loader,
            valid_loader,
            test_loader,
            max_epochs=epochs,
            test=True,
        )
    else:
        valid_metrics = [{'valid_dataset_iou': None}]
        test_metrics = [{'test_dataset_iou': None}]
    
    cust_model.eval()
    cust_save_path = os.path.join(
        WEIGHTS_DIR,
        f'replacedConv_{method_str}_{struct_str}_camvid_model_sp0{round(sparsity*10)}_fp32.pt'
    )
    save_load_torch_model(cust_model, cust_save_path, save=True)
    print(f'  Сохранено: {cust_save_path}')
    pruned_models[f'ReplacedConv_{method_str}_{struct_str}_sp0{round(sparsity*10)}'] = (cust_model, valid_metrics, test_metrics)

print(f'\n✓ Спарсификация завершена. Моделей: {len(pruned_models)}')


💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name    | Type     | Params | Mode 
---------------------------------------------
0 | model   | FPN      | 25.6 M | train
1 | loss_fn | DiceLoss | 0      | train
---------------------------------------------
25.6 M    Trainable params
0         Non-trainable params
25.6 M    Total params
102.357   Total estimated model params size (MB)
210       Modules in train mode
0         Modules in eval mode


СПАРСИФИКАЦИЯ И FINE-TUNING

Метод: IterativePruner (structured)
Original Conv2d:


Sanity Checking: |                                                                                            …

Training: |                                                                                                   …

Pruning step applied at epoch 0 start. Current sparsity: 0.1000


...

Validation: |                                                                                                 …

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


[{'valid_per_image_iou': 0.5965787768363953, 'valid_dataset_iou': 0.5960158705711365}]


Testing: |                                                                                                    …

[{'test_per_image_iou': 0.5163288116455078, 'test_dataset_iou': 0.5135128498077393}]
  Сохранено: weights/replacedConv_iterative_structured_camvid_model_sp05_fp32.pt

✓ Спарсификация завершена. Моделей: 2


In [18]:
print(pruned_models.keys())

dict_keys(['iterative_structured_sp05', 'ReplacedConv_iterative_structured_sp05'])


In [17]:
models_to_benchmark = []

In [18]:
# for model_name, (model, valid_metrics, test_metrics) in pruned_models.items():
#     models_to_benchmark.append((model_name, model, valid_metrics, test_metrics))

In [19]:
print('Загружаем базовые модели...')

valid_metrics, test_metrics, _ = train_val(
    original_model,
    train_loader,
    valid_loader,
    test_loader,
    max_epochs=5,
    train=False,
    test=True,
)

models_to_benchmark.append(('original_baseline', original_model, valid_metrics, test_metrics))

valid_metrics, test_metrics, _ = train_val(
    custom_model,
    train_loader,
    valid_loader,
    test_loader,
    max_epochs=5,
    train=False,
    test=True,
)

models_to_benchmark.append(('ReplacedConv_baseline', custom_model, valid_metrics, test_metrics))

💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Загружаем базовые модели...


Validation: |                                                                                                 …

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


[{'valid_per_image_iou': 0.849943995475769, 'valid_dataset_iou': 0.849575936794281}]


...

Validation: |                                                                                                 …

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


[{'valid_per_image_iou': 0.5913414359092712, 'valid_dataset_iou': 0.5902272462844849}]


Testing: |                                                                                                    …

[{'test_per_image_iou': 0.548675000667572, 'test_dataset_iou': 0.5446076989173889}]


In [20]:
print('Загружаем еще модели...')

more_model = CamVidModel("FPN", "resnext50_32x4d", in_channels=3, out_classes=OUT_CLASSES).to(DEVICE)

MORE_MODEL_PATH = os.path.join(WEIGHTS_DIR, 'originalConv_iterative_structured_camvid_model_sp05_fp32.pt')
if os.path.exists(MORE_MODEL_PATH):
    more_model.load_state_dict(torch.load(ORIGINAL_MODEL_PATH, map_location=DEVICE))
    print(f'Original модель загружена')
else:
    print(f'Original модель не найдена')

original_model.eval()

valid_metrics, test_metrics, _ = train_val(
    more_model,
    train_loader,
    valid_loader,
    test_loader,
    max_epochs=5,
    train=False,
    test=True,
)

models_to_benchmark.append(('iterative_structured_camvid_model_sp05_fp32', more_model, valid_metrics, test_metrics))

💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Загружаем еще модели...
Original модель загружена


Validation: |                                                                                                 …

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


[{'valid_per_image_iou': 0.849943995475769, 'valid_dataset_iou': 0.849575936794281}]


Testing: |                                                                                                    …

[{'test_per_image_iou': 0.784477174282074, 'test_dataset_iou': 0.7822542190551758}]


In [21]:
more_model = CamVidModel("FPN", "resnext50_32x4d", in_channels=3, out_classes=OUT_CLASSES).to(DEVICE)
replace_conv2d_with_custom(more_model)

MORE_MODEL_PATH = os.path.join(WEIGHTS_DIR, 'replacedConv_iterative_structured_camvid_model_sp05_fp32.pt')
if os.path.exists(MORE_MODEL_PATH):
    more_model.load_state_dict(torch.load(MORE_MODEL_PATH, map_location=DEVICE))
    print(f'Custom модель загружена')
else:
    print(f'Custom модель не найдена')

custom_model.eval()

valid_metrics, test_metrics, _ = train_val(
    more_model,
    train_loader,
    valid_loader,
    test_loader,
    max_epochs=5,
    train=False,
    test=True,
)

models_to_benchmark.append(('ReplacedConv_iterative_structured_camvid_model_sp05_fp32', more_model, valid_metrics, test_metrics))

💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Custom модель загружена


Validation: |                                                                                                 …

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


[{'valid_per_image_iou': 0.5965787768363953, 'valid_dataset_iou': 0.5960158705711365}]


Testing: |                                                                                                    …

[{'test_per_image_iou': 0.5163288116455078, 'test_dataset_iou': 0.5135128498077393}]


## 7. Бенчмаркинг

In [ ]:
WARM_N = 5
BENCH_N = 50

csv_filename = 'pruning_benchmark_results.csv'
writer = LatencyMetricsWriter(csv_filename)

benchmark_results = {}

for model_name, model, valid_metrics, test_metrics in models_to_benchmark:
    print(f'\nБенчмаркинг: {model_name}')
    model.eval()
    
    for bs in bs_range:
        input_batch = torch.ones([bs, 3, 128, 128], dtype=torch.float32, device=DEVICE)
        
        cpu_profiler = latency_cpu_profiler(
            model,
            input_batch,
            warmup_n=WARM_N,
            benchmark_n=BENCH_N
        )
        
        if DEVICE.type == 'cuda':
            gpu_mean, gpu_std = latency_gpu_event(
                model,
                input_batch,
                warmup=WARM_N,
                repeat=BENCH_N
            )
        else:
            gpu_mean, gpu_std = latency_cpu(
                model,
                input_batch,
                warmup_n=WARM_N,
                benchmark_n=BENCH_N
            )
        
        throughput = bs / (gpu_mean / 1000)
        
        # Запись метрик
        val_iou = valid_metrics[0]["valid_dataset_iou"]
        test_iou = test_metrics[0]["test_dataset_iou"]

        metrics = writer.record_metrics(
            model=model,
            model_name=model_name,
            batch_size=bs,
            precision=model_name.replace("ReplacedConv_", ""),
            input_shape=input_batch.shape,
            cpu_profiler=cpu_profiler,
            gpu_latency=(gpu_mean, gpu_std),
            notes=f"Pruning: {model_name}",
            val_iou=val_iou,
            test_iou=test_iou,
        )
        
        if model_name not in benchmark_results:
            benchmark_results[model_name] = []
        
        benchmark_results[model_name].append({
            'batch_size': bs,
            'latency_ms': gpu_mean,
            'throughput': throughput
        })
        
        print(f'  BS={bs}: {gpu_mean:.2f}ms ± {gpu_std:.2f}ms | {throughput:.1f} samples/s')

print(f'\n✓ Результаты сохранены в {csv_filename}')


Бенчмаркинг: original_baseline
Metrics recorded for original_baseline (batch_size=1)
  BS=1: 2.47ms ± 0.01ms | 405.3 samples/s
Metrics recorded for original_baseline (batch_size=2)
  BS=2: 2.72ms ± 0.02ms | 734.9 samples/s
Metrics recorded for original_baseline (batch_size=4)
  BS=4: 3.10ms ± 0.10ms | 1288.3 samples/s
Metrics recorded for original_baseline (batch_size=8)
  BS=8: 4.68ms ± 0.09ms | 1708.7 samples/s
Metrics recorded for original_baseline (batch_size=16)
  BS=16: 7.05ms ± 0.25ms | 2268.3 samples/s
Metrics recorded for original_baseline (batch_size=32)
  BS=32: 11.56ms ± 0.01ms | 2767.9 samples/s

Бенчмаркинг: ReplacedConv_baseline
Metrics recorded for ReplacedConv_baseline (batch_size=1)
  BS=1: 13.23ms ± 0.05ms | 75.6 samples/s
Metrics recorded for ReplacedConv_baseline (batch_size=2)
  BS=2: 13.54ms ± 0.08ms | 147.7 samples/s
Metrics recorded for ReplacedConv_baseline (batch_size=4)
  BS=4: 13.57ms ± 0.11ms | 294.8 samples/s


## 8. Анализ через analyze_benchmark_data()

In [ ]:
plots, summary, processed_df = analyze_benchmark_data('pruning_benchmark_results.csv')